In [1]:
import cv2 as cv
import numpy as np
import os
import matplotlib.pyplot as plt
# import matplotlib as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from PIL import Image
import glob as gb
from keras.utils import np_utils

### training information

In [2]:
def get_images(path, folder):
  labels = os.listdir(folder)[:62]
  data = []
  for label in labels:
    images = gb.glob(pathname=str(path + label + '/*.ppm'))
    class_num = labels.index(label)

    #************* Just for Tracing **********
    # image_array = cv.imread(images[0][0])
    # resized_img = cv.resize(image_array, (32, 32), interpolation=cv.INTER_AREA)   
    # cv.imshow('image', resized_img)
    # cv.waitKey(0)
    #*****************************************

    for image in images:
      image_array = cv.imread(f'{image}', cv.IMREAD_GRAYSCALE)
      resized_img = cv.resize(image_array, (32, 32), interpolation=cv.INTER_AREA)     
      data.append([resized_img, class_num])
  return data

In [3]:
train = get_images('Training/','Training')
test = get_images('Testing/','Testing')

def get_x_y(data):
  x, y = [], []
  for feature, label in data:
    x.append(feature)
    y.append(label)
  return x, y


In [4]:
x_train, y_train = get_x_y(train)
x_test, y_test = get_x_y(test)

In [5]:
x_train

[array([[ 42,  44,  50, ..., 131, 129, 128],
        [ 32,  35,  42, ..., 130, 126, 132],
        [ 33,  32,  28, ..., 152, 159, 164],
        ...,
        [160, 127,  86, ..., 118, 180, 199],
        [171, 125,  92, ...,  48, 137, 213],
        [122,  43,  38, ...,  44, 118, 209]], dtype=uint8),
 array([[ 52,  56,  45, ..., 114, 129, 127],
        [ 52,  50,  46, ..., 121, 132, 144],
        [ 52,  48,  44, ..., 167, 182, 186],
        ...,
        [ 82,  83,  67, ...,  73,  97,  99],
        [102, 118,  64, ...,  50,  99, 113],
        [115,  44,  34, ...,  47, 102, 125]], dtype=uint8),
 array([[ 57,  65,  62, ...,  73,  60,  49],
        [ 64,  64,  58, ...,  81,  60,  46],
        [ 68,  61,  53, ...,  80,  59,  44],
        ...,
        [157, 120,  89, ...,  77,  89,  85],
        [148, 107,  97, ...,  93,  92,  94],
        [128, 115, 107, ...,  87,  96,  95]], dtype=uint8),
 array([[ 62,  57,  52, ...,  46,  41,  45],
        [ 65,  60,  52, ...,  41,  40,  44],
        [ 61,  6

In [6]:
x_train = np.array(x_train) / 255
y_train = np.array(y_train)

In [7]:
x_test = np.array(x_test) / 255
y_test = np.array(y_test)

In [8]:
x_train = x_train.reshape(-1, 32, 32, 1)
x_train.shape

(4675, 32, 32, 1)

In [9]:
x_test = x_test.reshape(-1, 32, 32, 1)
x_test.shape

(2520, 32, 32, 1)

In [10]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [11]:
model = models.Sequential()
model.add(layers.Input(shape=(32, 32, 1)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(62, activation='softmax'))


In [12]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
callback = [tf.keras.callbacks.EarlyStopping(patience=25, monitor='val_accuracy')]

In [14]:
hist = model.fit(x_train, y_train, epochs=300, validation_split=0.05, validation_batch_size=8, batch_size=16, callbacks=callback)

Epoch 1/300
278/278 [==============================] - 2s 4ms/step - loss: 2.9221 - accuracy: 0.3267 - val_loss: 2.1822 - val_accuracy: 0.5684
Epoch 2/300
278/278 [==============================] - 1s 4ms/step - loss: 1.6065 - accuracy: 0.6145 - val_loss: 0.9289 - val_accuracy: 0.8248
Epoch 3/300
278/278 [==============================] - 1s 4ms/step - loss: 1.1112 - accuracy: 0.7203 - val_loss: 1.2883 - val_accuracy: 0.7564
Epoch 4/300
278/278 [==============================] - 1s 4ms/step - loss: 0.8619 - accuracy: 0.7791 - val_loss: 1.2565 - val_accuracy: 0.7906
Epoch 5/300
278/278 [==============================] - 1s 4ms/step - loss: 0.7107 - accuracy: 0.8174 - val_loss: 1.3028 - val_accuracy: 0.7650
Epoch 6/300
278/278 [==============================] - 1s 4ms/step - loss: 0.6399 - accuracy: 0.8250 - val_loss: 1.2476 - val_accuracy: 0.7778
Epoch 7/300
278/278 [==============================] - 1s 3ms/step - loss: 0.5340 - accuracy: 0.8489 - val_loss: 2.3361 - val_accuracy: 0.6368

In [15]:
loss, accuracy = model.evaluate(x_train, y_train)
print('Loss =', round(loss,3))
print(f'Accuracy = {round(accuracy*100,2)} %')

147/147 [==============================] - 0s 2ms/step - loss: 0.1450 - accuracy: 0.9730
Loss = 0.145
Accuracy = 97.3 %


In [16]:
loss, accuracy = model.evaluate(x_test, y_test)
print('Loss =', round(loss,3))
print(f'Accuracy = {round(accuracy*100,2)} %')

79/79 [==============================] - 0s 2ms/step - loss: 0.6772 - accuracy: 0.8683
Loss = 0.677
Accuracy = 86.83 %


### visualization

In [17]:
loss_train = hist.history['accuracy']
loss_val = hist.history['val_accuracy']
plt.plot(loss_train, 'g', label='Training accuracy')
plt.plot(loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

AttributeError: module 'matplotlib' has no attribute 'plot'

In [ ]:
loss_train = hist.history['loss']
loss_val = hist.history['val_loss']
plt.plot(loss_train, 'g', label='Training loss')
plt.plot(loss_val, 'b', label='validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
pred = model.predict(x_test)

In [ ]:
confusion_matrix(y_test.argmax(-1) ,pred.argmax(-1))

In [ ]:
cm = confusion_matrix(y_test.argmax(-1) ,pred.argmax(-1))
# plot confusion matrix
plot_confusion_matrix(conf_mat = cm,figsize=(15,15),
                     show_normed = True)

In [ ]:
from sklearn import metrics

fpr, tpr, thresholds = metrics.roc_curve(y_test.argmax(-1) ,pred.argmax(-1))
roc_auc = metrics.auc(fpr, tpr)

lw = 2
plt.figure()
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.15)
plt.plot(fpr, tpr, lw=lw, label=f'ROC curve (area = {roc_auc: 0.2f})')

plt.xlabel('(1-Specificity) - False Positive Rate')
plt.ylabel('Sensitivity - True Positive Rate')
plt.title(f'Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()